In [1]:
#-*- coding: utf-8 -*-
%matplotlib inline
from matplotlib.font_manager import FontProperties  
import matplotlib.pyplot as plt  
import numpy as np
import pandas as pd
import os

plt.rcParams['font.sans-serif'] = ['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号


/Users/ppj/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
taxi=pd.read_csv('/Users/ppj/Desktop/taxi_data/part-00000.csv',header=-1,names=['ID','warning','empty_taxi','dome_light'
                        ,'elevated','brake','received_time','GPS_time','Longitude','Latitude','speed','direction','satellite'])

In [3]:
taxi.GPS_time=taxi.GPS_time.map(lambda x: x[11:])

In [4]:
taxi_mor=taxi[(taxi.GPS_time>='07:30:00')&(taxi.GPS_time<='09:30:00')]
taxi_even=taxi[(taxi.GPS_time>='16:30:00')&(taxi.GPS_time<='19:00:00')]
taxi_midnight=taxi[(taxi.GPS_time>='00:00:00')&(taxi.GPS_time<='04:00:00')]
taxi_other=taxi[((taxi.GPS_time>='04:00:00')&(taxi.GPS_time<='07:30:00'))+((taxi.GPS_time>='09:30:00')&(taxi.GPS_time<='12:30:00'))+
               ((taxi.GPS_time>='19:00:00')&(taxi.GPS_time<='23:59:59'))]

/Users/ppj/anaconda/lib/python2.7/site-packages/pandas/computation/expressions.py:181: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  unsupported[op_str]))


In [5]:
taxi_se=taxi.loc[:,['ID','dome_light','elevated','GPS_time','Longitude','Latitude','speed']]
taxi_even_se=taxi_even.loc[:,['ID','dome_light','elevated','GPS_time','Longitude','Latitude','speed']]
taxi_mor_se=taxi_mor.loc[:,['ID','dome_light','elevated','GPS_time','Longitude','Latitude','speed']]
taxi_midnight_se=taxi_midnight.loc[:,['ID','dome_light','elevated','GPS_time','Longitude','Latitude','speed']]
taxi_other_se=taxi_other.loc[:,['ID','dome_light','elevated','GPS_time','Longitude','Latitude','speed']]

In [6]:
ID_unique=taxi_se.ID.unique()

In [7]:
def sort_time(group):
    return group.sort_index(by='GPS_time')

In [8]:
def taxi_group_id(taxi_se):
    taxi_filter=taxi_se[(taxi_se.dome_light=='0')+((taxi_se.dome_light=='1'))]
    #taxi_flag=taxi_filter.groupby('ID')
    taxi_flag_time=taxi_filter.sort_index(by='GPS_time')
    return taxi_flag_time

In [9]:
def get_speed_information(taxi_id,taxi_flag_time):
    s=taxi_flag_time[taxi_flag_time.ID==taxi_id]
    speed=np.array(s.speed)
    speed_list=speed[speed!=0]
    speed_mean=speed_list.mean()    #均值
    speed_std=speed_list.std()      #标准差
    #speed_max=speed_list.max()       #速度最大
    #speed_min=speed_list.min()       #速度最小
    
    #plt.plot(speed_list)
    #plt.title(u'司机:%s 的历史速度变化图'%taxi_id)
    #plt.savefig('speed_%s' %taxi_id)
    return [speed_mean,speed_std]

In [18]:
def taxi_speed_analysis(taxi_id):
   
    id_time=taxi_group_id(taxi_se)
    speed_id_time=np.array(id_time[id_time.ID==taxi_id].speed)
    speed_list=speed_id_time[speed_id_time!=0]
    if len(speed_list)>20:
        id_even_time=taxi_group_id(taxi_even_se)
        id_mor_time=taxi_group_id(taxi_mor_se)
        id_midnight_time=taxi_group_id(taxi_midnight_se)
        id_other_time=taxi_group_id(taxi_other_se)
        collect_list=[id_time,id_even_time,id_mor_time,id_midnight_time,id_other_time]
        collect_info=[]
        for i in range(len(collect_list)):
            if taxi_id in list(collect_list[i].ID):
                speed_information=get_speed_information(taxi_id,collect_list[i])
                collect_info.append([speed_information,i])
    
        flag_max=1
        flag_min=1
        min_std=collect_info[1][0][1]
        max_std=collect_info[1][0][1]
        for j in range(1,len(collect_info)):
            if collect_info[j][0][1]<=min_std:
                min_std=collect_info[j][0][1]
                flag_min=j
            if collect_info[j][0][1]>=max_std:
                max_std=collect_info[j][0][1]
                flag_max=j
        time_flag=['nan','evening','morning','midnight','other_time']   #时间段标志
        #print u'司机:%s 的历史平均速度: %f'%(taxi_id,collect_info[0][0][0])
        #print u'司机:%s 的历史速度方差: %f'%(taxi_id,collect_info[0][0][1])
        #print u'司机:%s 的开车最稳在:%s'%(taxi_id,time_flag[collect_info[flag_max][1]])
        #print u'司机:%s 的开车最不稳在:%s'%(taxi_id,time_flag[collect_info[flag_min][1]])
        '''
        fig = plt.figure()
        ax=fig.add_subplot(1,1,1)
        plt.plot(speed_list)
        plt.title(u'Driver:%s the change of historical rate'%taxi_id)
        plt.ylabel(u'Speed: km/h')
        plt.savefig('speed_%s.jpg' %taxi_id)
        '''
        return [taxi_id,collect_info[0][0][0],collect_info[0][0][1],time_flag[collect_info[flag_max][1]]
                ,time_flag[collect_info[flag_min][1]]]
    else:
        return [np.nan]*5

In [19]:
taxi_speed_analysis(ID_unique[1])

/Users/ppj/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


[27320, 32.175862068965515, 24.012247667595396, 'other_time', 'evening']

In [20]:
collect_end=[]
for x in ID_unique[0:]:
    k=taxi_speed_analysis(x)
    collect_end.append(k)

    
    

/Users/ppj/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


KeyboardInterrupt: 

In [21]:
collect_end

[[18834, 41.161290322580648, 19.452335085293786, 'other_time', 'evening'],
 [27320, 32.175862068965515, 24.012247667595396, 'other_time', 'evening'],
 [16926, 41.799999999999997, 24.429618294193631, 'morning', 'evening'],
 [26337, 30.156862745098039, 13.589748946619341, 'other_time', 'evening'],
 [23807, 36.598076923076924, 21.451846705377275, 'other_time', 'morning'],
 [23390, 31.351063829787229, 15.506757241437098, 'other_time', 'morning'],
 [10827, 5.4142857142857137, 2.979778789214572, 'other_time', 'evening'],
 [20043, 41.527777777777779, 19.606526848519817, 'other_time', 'evening'],
 [nan, nan, nan, nan, nan],
 [21941, 43.513636363636358, 24.041547915448351, 'morning', 'evening'],
 [15042, 23.574999999999996, 16.229283803599522, 'evening', 'other_time'],
 [27615, 29.865116279069763, 20.915109523543688, 'morning', 'evening'],
 [14891, 27.49074074074074, 15.75334495366241, 'morning', 'evening'],
 [14230, 32.214999999999996, 19.322144679098127, 'other_time', 'evening'],
 [15990, 25.

In [22]:
pd.DataFrame(collect_end,columns=['ID ','Historical_rate','historical_std','max_steady','min_steady']).to_csv('driver.csv')